In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor, optim, cuda
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.nn as nn
import csv
import os
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor, optim, cuda
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import csv
import os

from data.data import DeClareDataset

In [2]:
torch.__version__

'1.0.0'

# Load Datasets

In [3]:
SNOPES_LOC = "./Datasets/Snopes/snopes.tsv"
#consists of rumors analyzed on the Snopes website along with their credibility labels (true or false), 
#sets of reporting articles, and their respective web sources

POLITIFACT_LOC = "./Datasets/PolitiFact/politifact.tsv"

glove_data_file = "./Glove/glove.6B.100d.txt"

In [4]:
snopes = DeClareDataset(SNOPES_LOC, glove_data_file)

Successfully read news data from ./Datasets/Snopes/snopes.tsv
Number of articles = 29242
Number of claims = 4341
Using pre-built vocabulary


## Preliminary Analysis

In [5]:
snopes.news_df.head()

,Credibility,Claim_Source,Claim,Article,Article_Source
0,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,there are several holidays throughout that tim...,www.godlikeproductions.com
1,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,defenses against same photographs show images ...,www.sjpba.net
2,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,the best that life could think out we extended...,www.englisher.net
3,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,this entry november 19 2006 published 9 years ...,rss2.com
4,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,place he will not do either said snape the ord...,www.englisher.net


In [6]:
snopes.glove_df.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
the,-0.038194,-0.244870,0.72812,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,0.087459,...,0.016215,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.520280,-0.14590,0.82780,0.270620
",",-0.107670,0.110530,0.59812,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,-0.094189,...,0.349510,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.351110,-0.83155,0.45293,0.082577
.,-0.339790,0.209410,0.46348,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,-0.019169,...,-0.063351,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.392420,-0.23394,0.47298,-0.028803
of,-0.152900,-0.242790,0.89837,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,0.425410,...,0.187120,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.560940,-0.59100,1.00390,0.206640
to,-0.189700,0.050024,0.19084,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,0.342410,...,-0.131340,0.058617,-0.318690,-0.61419,-0.62393,-0.41548,-0.038175,-0.39804,0.47647,-0.159830


In [107]:
class DeClareModel(nn.Module):
    def __init__(self, glove_embeddings, claim_source_vocab_size, article_source_vocab_size, nb_lstm_units, batch_size):
        super(DeClareModel, self).__init__()
        
        self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(glove_embeddings), freeze=False)
        self.claim_source_embeddings = nn.Embedding(claim_source_vocab_size, claim_source_vocab_size)
        self.article_source_embeddings = nn.Embedding(article_source_vocab_size, article_source_vocab_size)
        
        self.batch_size = batch_size
        self.embedding_dim = glove_embeddings.shape[1]
        self.nb_lstm_units = nb_lstm_units
        
        self.dense = nn.Linear(2*self.embedding_dim, 1)
        
        self.biLSTM = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.nb_lstm_units,
                              batch_first=True, bidirectional=True)
        
    def forward(self, claim, claim_len, article, article_len, claim_source, article_source):
        
        self.hidden = self.init_hidden()
        
        #claim_embeddings = self.word_embeddings(claim).float()
        claim_mean_embedding = torch.mean(self.word_embeddings(claim), 1)
        #shape : (batch, embedding_dim)
        
        article_embeddings = self.word_embeddings(article).float()
        #shape : (batch, seq_len, embedding_dim)
        
        # top network
        claim_article_concat = torch.cat([article_embeddings, 
                                          claim_mean_embedding.unsqueeze(1).expand(-1, article_embeddings.shape[1], -1)], 2)
        #shape : (batch, seq_len, 2*embedding_dim)
        
        claim_article_concat = F.tanh(self.dense(claim_article_concat)).squeeze()
        #shape : (batch, seq_len)
        
        claim_article_concat = F.softmax(claim_article_concat)
        #shape : (batch, seq_len)
        # this could be problematic because seq_len will have the padded stuff and unlike others functions
        # softmax is not point-wise 
        
        
        # bottom network
        
        A = nn.utils.rnn.pack_padded_sequence(article_embeddings, article_len, batch_first=True)
        
        A, self.hidden = self.biLSTM(A, self.hidden)
        
        A, _ = torch.nn.utils.rnn.pad_packed_sequence(A, batch_first=True, padding_value=0)
        #shape : (batch_size, seq_len, num_dir*hidden_units)
        
        #demo_output = torch.cat((claim_mean_embedding.view(1,-1), article_embeddings), 0)
        return A
    
    def init_hidden(self):
        # the weights are of the form (num_layers*num_directions, batch, hidden_size)
        hidden_a = torch.randn(1*2, self.batch_size, self.nb_lstm_units)
        hidden_b = torch.randn(1*2, self.batch_size, self.nb_lstm_units)

        #hidden_a = Variable(hidden_a)
        #hidden_b = Variable(hidden_b)

        return (hidden_a, hidden_b)

In [108]:
batch_size = 4
nb_lstm_units = 64

In [109]:
dataloader = DataLoader(snopes, batch_size)

In [110]:
declare = DeClareModel(snopes.initial_embeddings, snopes.claim_source_vocab_size, 
                       snopes.article_source_vocab_size, nb_lstm_units, batch_size)

In [111]:
for data_sample in dataloader:
    temp = declare(data_sample[0], data_sample[1], data_sample[2], data_sample[3], data_sample[4], data_sample[5])
    break

In [112]:
temp.shape

torch.Size([4, 100, 128])

In [138]:
news_df = pd.read_csv(SNOPES_LOC, sep='\t', header=None)
news_df.columns = ['Credibility', 'Claim_Source', 'Claim', 'Article', 'Article_Source']

In [99]:
news_df['Claim'].iloc[0]

'best buy chain eschewing use word christmas 2006 holiday print advertising'

In [137]:
news_df['Article'].str.split().str.len().min()

21

In [24]:
snopes._vec('there')

array([-0.084215 ,  0.69597  ,  0.28383  , -0.22497  , -0.55923  ,
        0.21196  , -0.15175  ,  0.31601  ,  0.24803  , -0.36385  ,
        0.2252   ,  0.34109  ,  0.61438  ,  0.08318  ,  0.74894  ,
       -0.38785  , -0.27211  ,  0.2656   , -0.66332  ,  0.45571  ,
        0.34949  ,  0.38635  ,  0.25707  , -0.60101  , -0.047292 ,
       -0.41198  ,  0.38246  , -0.52151  ,  0.037757 , -0.42492  ,
       -0.36352  , -0.037377 ,  0.31263  ,  0.19709  ,  0.008142 ,
        0.53871  , -0.074505 ,  0.31395  ,  0.35129  , -0.39005  ,
       -0.5474   , -0.11395  ,  0.076668 , -0.61069  ,  0.15894  ,
       -0.33043  ,  0.75967  , -0.52289  , -0.38957  , -0.71164  ,
        0.28724  , -0.35683  ,  0.050529 ,  1.3392   , -0.16064  ,
       -2.9236   ,  0.17524  , -0.42109  ,  1.5235   ,  0.85181  ,
       -0.47563  ,  1.1225   , -0.48463  ,  0.36458  ,  0.97809  ,
       -0.2227   ,  0.88791  ,  0.068738 ,  0.22557  ,  0.026459 ,
       -0.032799 , -0.40426  ,  0.14579  , -0.4535   ,  0.4775

In [132]:
a = torch.ones(5, 2, 4)
b = torch.randn(5, 4)
#c shape (5, 2, 8)

In [133]:
c = torch.cat([a,b.unsqueeze(1).expand(-1, 2, -1)], 2)

In [134]:
a, b

(tensor([[[1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.]]]), tensor([[ 0.3675, -1.1802,  0.7398,  1.7697],
         [-1.6448,  0.2826, -0.9997,  1.0599],
         [ 1.1018, -1.7394, -1.5658, -0.7811],
         [ 0.5521,  0.8893,  1.4350,  0.6763],
         [-0.4845, -0.4943,  0.2353,  0.0253]]))

In [135]:
c

tensor([[[ 1.0000,  1.0000,  1.0000,  1.0000,  0.3675, -1.1802,  0.7398,
           1.7697],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  0.3675, -1.1802,  0.7398,
           1.7697]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000, -1.6448,  0.2826, -0.9997,
           1.0599],
         [ 1.0000,  1.0000,  1.0000,  1.0000, -1.6448,  0.2826, -0.9997,
           1.0599]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000,  1.1018, -1.7394, -1.5658,
          -0.7811],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.1018, -1.7394, -1.5658,
          -0.7811]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000,  0.5521,  0.8893,  1.4350,
           0.6763],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  0.5521,  0.8893,  1.4350,
           0.6763]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000, -0.4845, -0.4943,  0.2353,
           0.0253],
         [ 1.0000,  1.0000,  1.0000,  1.0000, -0.4845, -0.4943,  0.2353,
           0.0253]]])

In [139]:
news_df.head()

,Credibility,Claim_Source,Claim,Article,Article_Source
0,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,there are several holidays throughout that tim...,www.godlikeproductions.com
1,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,defenses against same photographs show images ...,www.sjpba.net
2,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,the best that life could think out we extended...,www.englisher.net
3,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,this entry november 19 2006 published 9 years ...,rss2.com
4,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,place he will not do either said snape the ord...,www.englisher.net


In [143]:
news_df[news_df['Credibility'].isin(['true'])]

,Credibility,Claim_Source,Claim,Article,Article_Source
0,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,there are several holidays throughout that tim...,www.godlikeproductions.com
1,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,defenses against same photographs show images ...,www.sjpba.net
2,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,the best that life could think out we extended...,www.englisher.net
3,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,this entry november 19 2006 published 9 years ...,rss2.com
4,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,place he will not do either said snape the ord...,www.englisher.net
5,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,at the same time that it the head 1806 5 13 ba...,www.englisher.net
31,true,politics_guns_bushnra,1995 former president george hw bush resigned ...,young man with a history of mental disturbance...,www.washingtonpost.com
32,true,politics_guns_bushnra,1995 former president george hw bush resigned ...,timothy mcveigh bombed the alfred p murrah fed...,thenewcivilrightsmovement.com
33,true,politics_guns_bushnra,1995 former president george hw bush resigned ...,the gun control advocacy groups the brady camp...,en.wikipedia.org
34,true,politics_guns_bushnra,1995 former president george hw bush resigned ...,bush s letter of resignation from the nra dece...,www.addictinginfo.org
